In [1]:
import pymysql
from matplotlib.backends.backend_agg import RendererAgg
import numpy as np
import pandas as pd
import xmltodict
from pandas import json_normalize
import urllib.request
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from PIL import Image
#import gender_guesser.detector as gender
import requests
import base64
import requests
from bs4 import BeautifulSoup
import re
import json
from multiprocessing import Pool
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from functools import reduce
import time

In [4]:
conn = pymysql.connect(host='database-1.ciwdlxclxwk2.eu-west-1.rds.amazonaws.com', 
                       user='admin', 
                       password='12345678')

cursor = conn.cursor()
cursor.execute("select database();")
cursor.connection.commit()
sql = '''use postcodes'''
cursor.execute(sql)

sql = "select * from properties"
cursor.execute(sql)

existing = pd.DataFrame.from_records(cursor.fetchall(), columns=['address', 'propertyType', 'bedrooms', 'bathrooms',
       'transactions_price', 'transactions_date', 'transactions_tenure', 'lat',
       'lgt', 'detailUrl', 'transactions_date_dt', 'postcode'])
existing

,address,propertyType,bedrooms,bathrooms,transactions_price,transactions_date,transactions_tenure,lat,lgt,detailUrl,transactions_date_dt,postcode
0,"Apartment 9, Old Mile House Court, St Albans, ...",Flat,2.0,None,"£477,500",5 Jul 2019,Leasehold,51.73966,-0.31749,https://www.rightmove.co.uk/house-prices/detai...,2019-07-05,al11ar
1,"Apartment 9, Old Mile House Court, St Albans, ...",Flat,2.0,None,"£430,000",14 Aug 2014,Leasehold,51.73966,-0.31749,https://www.rightmove.co.uk/house-prices/detai...,2014-08-14,al11ar
2,"Apartment 8, Old Mile House Court, St Albans, ...",Flat,2.0,None,"£387,500",15 Mar 2019,Leasehold,51.73966,-0.31749,https://www.rightmove.co.uk/house-prices/detai...,2019-03-15,al11ar
3,"Apartment 8, Old Mile House Court, St Albans, ...",Flat,2.0,None,"£325,000",2 Oct 2014,Leasehold,51.73966,-0.31749,https://www.rightmove.co.uk/house-prices/detai...,2014-10-02,al11ar
4,"Apartment 3, Old Mile House Court, St Albans, ...",Flat,2.0,None,"£375,000",26 Jan 2018,Leasehold,51.73966,-0.31749,https://www.rightmove.co.uk/house-prices/detai...,2018-01-26,al11ar
...,...,...,...,...,...,...,...,...,...,...,...,...
22971,"9, Cromwell Close, St. Albans, Hertfordshire A...",Detached,NaN,None,"£665,000",3 Jul 2015,Freehold,51.77354,-0.29739,,2015-07-03,al49ye
22972,"12, Cromwell Close, St. Albans, Hertfordshire ...",Detached,4.0,None,"£650,000",9 Apr 2015,Freehold,51.77354,-0.29739,https://www.rightmove.co.uk/house-prices/detai...,2015-04-09,al49ye
22973,"12, Cromwell Close, St. Albans, Hertfordshire ...",Detached,4.0,None,"£480,000",14 Oct 2011,Freehold,51.77354,-0.29739,https://www.rightmove.co.uk/house-prices/detai...,2011-10-14,al49ye
22974,"3, Cromwell Close, St Albans, Hertfordshire AL...",Detached,3.0,None,"£472,500",25 Oct 2010,Freehold,51.77354,-0.29739,https://www.rightmove.co.uk/house-prices/detai...,2010-10-25,al49ye


In [20]:
postcode_s = 'AL1 390'
postcode_s = postcode_s.replace(" ","").lower()

if len(postcode_s)<6:
    sql = "select * from properties where postcode like '{}%'".format(postcode_s)
else:
    sql = "select * from properties where postcode ='{}'".format(postcode_s)

